# deep nn with dropout

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets('../data/', one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


# define model

In [2]:
n_input = 28 * 28 * 1 ## n_height = n_width = 28, n_c = 1
n_l1 = 128 ## layer 1 node count

n_classes = 10 ## the output class count, the softmax output

x = tf.placeholder(tf.float32, shape=[None, n_input])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
stddev = 0.05
w1 = tf.Variable(tf.random_normal([n_input, n_l1], stddev=stddev))
b1 = tf.Variable(tf.zeros([1, n_l1]))

w2 = tf.Variable(tf.random_normal([n_l1, n_classes], stddev=stddev))
b2 = tf.Variable(tf.zeros([1, n_classes]))

print "NETWORK ready!"

NETWORK ready!


In [3]:
def z(X, W, b):
    return tf.add(tf.matmul(X, W) , b)
## define the graph
z1 = z(x, w1, b1)
a1 = tf.nn.sigmoid(z1)

z2 = z(a1, w2, b2)
y_ = tf.nn.softmax(z2)

learning_rate = 0.001

# cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))
# optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
optm = tf.train.AdamOptimizer().minimize(cost)
corr = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))

# initialize
init = tf.global_variables_initializer()
print "gradient decent is ready"

gradient decent is ready


In [5]:
# run it!
epochs = 10
batch_size      = 128
display_step    = 1
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
print "init run done"

total_batch = int(mnist.train.num_examples/batch_size)
print "total batch ", total_batch

# Summary
writer = tf.summary.FileWriter('/tmp/tf_logs/mlp-mnist', graph=sess.graph)
tf.summary.scalar("cost_per_batch", cost)
tf.summary.scalar("accuracy", accr)
ss = tf.summary.merge_all()

# OPTIMIZE
j = 0;
for epoch in range(epochs):
    avg_cost = 0.
    print "#Epoch ", epoch
    
    # ITERATION
    for i in range(total_batch):
        j += 1
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        _, summary, percost = sess.run([optm, ss, cost], feed_dict=feeds)
        avg_cost += percost
        writer.add_summary(summary, j)
    avg_cost = avg_cost / total_batch
    
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
                
print ("OPTIMIZATION FINISHED")

init run done
total batch  429
#Epoch  0
Epoch: 000/010 cost: 1.805062529
TRAIN ACCURACY: 0.781
TEST ACCURACY: 0.848
#Epoch  1
Epoch: 001/010 cost: 1.578660777
TRAIN ACCURACY: 0.945
TEST ACCURACY: 0.926
#Epoch  2
Epoch: 002/010 cost: 1.547456722
TRAIN ACCURACY: 0.953
TEST ACCURACY: 0.937
#Epoch  3
Epoch: 003/010 cost: 1.534358830
TRAIN ACCURACY: 0.938
TEST ACCURACY: 0.942
#Epoch  4
Epoch: 004/010 cost: 1.525526925
TRAIN ACCURACY: 0.945
TEST ACCURACY: 0.947
#Epoch  5
Epoch: 005/010 cost: 1.518840484
TRAIN ACCURACY: 0.977
TEST ACCURACY: 0.951
#Epoch  6
Epoch: 006/010 cost: 1.513697762
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.954
#Epoch  7
Epoch: 007/010 cost: 1.509312522
TRAIN ACCURACY: 0.977
TEST ACCURACY: 0.955
#Epoch  8
Epoch: 008/010 cost: 1.505706831
TRAIN ACCURACY: 0.961
TEST ACCURACY: 0.958
#Epoch  9
Epoch: 009/010 cost: 1.502539490
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.960
OPTIMIZATION FINISHED
